## Observations and Insights 

In [1]:
%matplotlib widget

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
scc_combined_df = pd.DataFrame.merge(mouse_metadata,study_results,on="Mouse ID",how="outer")

# Display the data table for preview
#scc_data_df.info()
scc_combined_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [3]:
# Checking the number of mice.
scc_combined_df["Mouse ID"].nunique()

249

In [4]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 

In [5]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mice = scc_combined_df[scc_combined_df.duplicated(["Mouse ID","Timepoint"])]
duplicate_mice

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.000000,0
911,g989,Propriva,Female,21,26,5,47.570392,0
913,g989,Propriva,Female,21,26,10,49.880528,0
915,g989,Propriva,Female,21,26,15,53.442020,0
917,g989,Propriva,Female,21,26,20,54.657650,1


In [6]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
scc_cleaned_df = scc_combined_df.drop_duplicates(["Mouse ID","Timepoint"])
scc_cleaned_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [7]:
# Checking the number of mice in the clean DataFrame.
num_mice = scc_cleaned_df["Mouse ID"].nunique()
num_mice

249

## Summary Statistics

In [8]:
scc_cleaned_df["Drug Regimen"].value_counts()

Capomulin    230
Ramicane     228
Ketapril     188
Naftisol     186
Zoniferol    182
Placebo      181
Stelasyn     181
Infubinol    178
Ceftamin     178
Propriva     156
Name: Drug Regimen, dtype: int64

In [9]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method is the most straightforward, creating multiple series and putting them all together at the end.

# tumor_volume_ramicane = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Ramicane"),"Tumor Volume (mm3)"]

# tumor_volume_ramicane.mean()
# 40.67323554

In [10]:
# # Do the above with conditional loops?

# # Create a list of regimens
# regimen_list = scc_cleaned_df["Drug Regimen"].value_counts()

# # Create variables to hold data for each drug regimen
# drugs = []
# means = []
# median = []
# variance = []
# standard_deviation = []
# sem = []

# # Use a loop to create a series of tumor volume for each drug regimen
# drug_regimen = scc_cleaned_df["Drug Regimen"]

# for drug in drug_regimen:
#         means.append((scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Ramicane"),"Tumor Volume (mm3)"]).mean())
        

In [11]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# This method produces everything in a single groupby function
scc_cleaned_df.groupby(["Drug Regimen"]).agg({"Tumor Volume (mm3)": ["mean","median","var","std","sem"]}).round(2)

Tumor Volume (mm3)                          
                           mean median    var   std   sem
Drug Regimen                                             
Capomulin                 40.68  41.56  24.95  4.99  0.33
Ceftamin                  52.59  51.78  39.29  6.27  0.47
Infubinol                 52.88  51.82  43.13  6.57  0.49
Ketapril                  55.24  53.70  68.55  8.28  0.60
Naftisol                  54.33  52.51  66.17  8.13  0.60
Placebo                   54.03  52.29  61.17  7.82  0.58
Propriva                  52.39  50.91  43.14  6.57  0.53
Ramicane                  40.22  40.67  23.49  4.85  0.32
Stelasyn                  54.23  52.43  59.45  7.71  0.57
Zoniferol                 53.24  51.82  48.53  6.97  0.52

## Bar and Pie Charts

In [12]:
drug_mouse_count = scc_cleaned_df.groupby(["Drug Regimen"],as_index=False).agg({"Mouse ID": pd.Series.nunique})
drug_mouse_count

,Drug Regimen,Mouse ID
0,Capomulin,25
1,Ceftamin,25
2,Infubinol,25
3,Ketapril,25
4,Naftisol,25
5,Placebo,25
6,Propriva,25
7,Ramicane,25
8,Stelasyn,24
9,Zoniferol,25


In [13]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
# Group data by drug regimen and aggregate by number of mice (count)
# mice_by_regimen_df = scc_cleaned_df.groupby(["Drug Regimen"],as_index=False).agg({"Mouse ID": "count"})
# Plot data as a bar chart
bar_plot = drug_mouse_count.plot(kind="bar",x="Drug Regimen",y="Mouse ID",legend=False)
bar_plot.set_xlabel("Drug Regimen") # x axis labels
bar_plot.set_ylabel("Number of mice") # y axis labels
plt.tight_layout() # Fit plot to space

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.
plt.figure()
# fig1,ax1 = plt.subplots()
drug_list = drug_mouse_count["Drug Regimen"].unique() # Add categories for x axis
x_axis = np.arange(0, len(drug_list)) # x axis length
num_mice = drug_mouse_count["Mouse ID"] # y axis values
tick_locations = [value for value in x_axis] # Add tick locations
plt.xticks(tick_locations,drug_list,rotation=90) # Add x tick labels
plt.xlabel("Drug Regimen") # x axis labels
plt.ylabel("Number of mice") # y axis labels
plt.tight_layout() # Fit plot to space
plt.bar(x_axis,num_mice)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 10 artists>

In [15]:
sex_mouse_count = scc_cleaned_df.groupby(["Sex"],as_index=False).agg({"Mouse ID": pd.Series.nunique})
sex_mouse_count = sex_mouse_count.rename(columns={"Mouse ID":"Number of mice"})
sex_mouse_count

,Sex,Number of mice
0,Female,124
1,Male,125


In [16]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
pie_plot = sex_mouse_count.plot(kind="pie",y="Number of mice", labels=sex_mouse_count["Sex"],autopct='%1.1f%%',startangle=135,legend=False)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# # Generate a pie plot showing the distribution of female versus male mice using pyplot
labels = sex_mouse_count["Sex"]
sizes = sex_mouse_count["Number of mice"]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=135)
plt.ylabel("Number of mice")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Quartiles, Outliers and Boxplots

In [19]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Create data frames for each relevant drug regimen
capomulin_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Capomulin")]
ramicane_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Ramicane")]
infubinol_df = scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Infubinol")]
ceftamin_df =  scc_cleaned_df.loc[(scc_cleaned_df["Drug Regimen"]=="Ceftamin")]
# Concatenate individual drug regimen data frames
treatment_df = pd.concat([capomulin_df,ramicane_df,infubinol_df,ceftamin_df],ignore_index=True)
# treatment_df
# Start by getting the last (greatest) timepoint for each mouse
last_timepoint_df = treatment_df.groupby(["Mouse ID"],as_index=False).agg({"Timepoint" : pd.Series.max})
# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
merged_df = pd.merge(last_timepoint_df,treatment_df,on="Mouse ID",how="left")
# Keep rows with final timepoint only
merged_df2 = merged_df.loc[merged_df["Timepoint_y"]==merged_df["Timepoint_x"]]
# Create new data frame with relvant data only
final_tumor_df = merged_df2[["Mouse ID","Timepoint_x","Drug Regimen","Sex","Age_months","Weight (g)","Tumor Volume (mm3)","Metastatic Sites"]]
# Rename column
final_tumor_df = final_tumor_df.rename(columns={"Timepoint_x":"Timepoint"})
# Reset index
final_tumor_df.reset_index(drop=True,inplace=True)
# final_tumor_df

In [20]:
# Put treatments into a list for for loop (and later for plot labels)
treatment_list = final_tumor_df["Drug Regimen"].unique()
# treatment_list
# Create empty list to fill with tumor vol data (for plotting)
tumor_vol_data = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for treatment in treatment_list:
        quartiles = final_tumor_df.loc[(final_tumor_df["Drug Regimen"]==treatment)]["Tumor Volume (mm3)"].quantile([.25,.5,.75])
        lowerq = quartiles[0.25]
        upperq = quartiles[0.75]
        iqr = round(upperq-lowerq,2)
        lower_bound = round(lowerq - (1.5*iqr),2)
        upper_bound = round(upperq + (1.5*iqr),2)
        # Locate the rows which contain mice on each drug and get the tumor volumes
        tumor_vol_data.append(final_tumor_df.loc[(final_tumor_df["Drug Regimen"]==treatment)]["Tumor Volume (mm3)"])
        
        # add subset 
        # ???
        
        # Determine outliers using upper and lower bounds
        print(f"The interquartile range for {treatment} is {iqr}")
        print(f"Values below {lower_bound} could be outliers.")
        print(f"Values above {upper_bound} could be outliers.")


The interquartile range for Infubinol is 11.48
Values below 36.83 could be outliers.
Values above 82.75 could be outliers.
The interquartile range for Ceftamin is 15.58
Values below 25.35 could be outliers.
Values above 87.67 could be outliers.
The interquartile range for Ramicane is 9.1
Values below 17.91 could be outliers.
Values above 54.31 could be outliers.
The interquartile range for Capomulin is 7.78
Values below 20.71 could be outliers.
Values above 51.83 could be outliers.


In [23]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

fig1, ax1 = plt.subplots()
# ax1.set_title('Temperatures at LAX')
# ax1.set_ylabel('Temperature (°F)')
ax1.boxplot(tumor_vol_data)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin




In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
